# FPS Correction
The FPS for all motions in SMPLX is currently set to 120, which is incorrect and needs to be adjusted. Meanwhile, the FPS for SMPLH is correct, so we need to align all SMPLX FPS values with the corresponding SMPLH FPS values.

The process consists of two steps:

1. Standardize file names to ensure correspondence, as there are slight differences between the downloaded SMPLH and SMPLX file names.
   
2. Adjust the FPS for SMPLX files based on their corresponding SMPLH files.
The detailed code is provided below.



## 1. Renaming Instructions
The following code is used to rename SMPLX file names to match those of SMPLH based on SMPLH naming conventions. Note that both SMPLH and SMPLX files end with .npz. Also, please note that this code only needs to be executed once.

Finally, you'll notice that the total number of SMPLX files will be slightly less than the SMPLH files. This is because some CMU dataset files are not included in SMPLX.

In [ ]:
import re
import shutil
import glob
import os


import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin


smplh_file_list = []
smplh_file_replace_list = []
smplh_path = '../datasets/AMASS_SMPLH/'
smplh_search_path = os.path.join(smplh_path, '**', '*.npz')
smplh_npz_files = glob.glob(smplh_search_path, recursive=True)
for smplh_name in smplh_npz_files:
    if 'BMLmovi' in smplh_name or 'BMLhandball' in smplh_name or 'DFaust_67' in smplh_name: continue
    name = smplh_name[len(smplh_path):]
    smplh_file_list.append(name)
    smplh_file_replace_list.append(name.replace(' ','_'))
print(len(smplh_file_list))

smplx_file_list = []
smplx_path = '../datasets/AMASS_SMPLX/'
smplx_search_path = os.path.join(smplx_path, '**', '*.npz')
smplx_npz_files = glob.glob(smplx_search_path, recursive=True)
tot = 0
for name in smplx_npz_files:
    if 'male_stagei.npz' in name or 'female_stagei.npz' in name:
        continue
    relpath = name[len(smplx_path):]
    relpath = relpath[:-12]+'_poses.npz'  # Replace the ending _stageii.npz with _poses.npz
    if relpath not in smplh_file_replace_list:
        print(relpath)
        continue
    
    for index,smplh_replace_name in enumerate(smplh_file_replace_list):
        if relpath in smplh_replace_name:
            shutil.move(name,smplx_path+smplh_file_list[index])
    
    tot = tot+1
    smplx_file_list.append(relpath)
print(tot)

## 2. FPS Correction
The FPS for all motions in SMPLX is set to 120, which is incorrect and needs to be corrected to synchronize with SMPLH. (The FPS of SMPLH is right)

The code for correction is provided below.

In [1]:
import re
import shutil
import glob
import os


import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin



smplh_path = '../datasets/AMASS_SMPLH/'

smplx_path = '../datasets/AMASS_SMPLX/'
smplx_search_path = os.path.join(smplx_path, '**', '*.npz')
smplx_npz_files = glob.glob(smplx_search_path, recursive=True)
for name in smplx_npz_files:
    if 'male_stagei.npz' in name or 'female_stagei.npz' in name:
        continue
    relpath = name[len(smplx_path):]
    smplx_file = np.load(name,allow_pickle=True)
    smplx_data_dict = {key: smplx_file[key] for key in smplx_file}
    
    smplh_file = np.load(smplh_path+relpath,allow_pickle=True)
    
    assert 'mocap_frame_rate' in smplx_file.files
    assert 'mocap_framerate' in smplh_file.files
    
    smplx_data_dict['mocap_frame_rate'] = smplh_file['mocap_framerate']
    np.savez(name, **smplx_data_dict)


11370
